In [14]:
import sys
sys.path.append("../src")
import climber_simulator as cs
import pandas as pd

In [15]:
casual = []
skilled = []
elite = []

# build 100 data for each category
for _ in range(100):
    casual.append(cs.simulator('casual'))
    skilled.append(cs.simulator('skilled'))
    elite.append(cs.simulator('elite'))

df_casual = pd.DataFrame(casual)
df_skilled = pd.DataFrame(skilled)
df_elite = pd.DataFrame(elite)


In [3]:
df_casual.to_csv("casual_dataset.csv",index=False)
df_skilled.to_csv("skilled_dataset.csv",index=False)
df_elite.to_csv("elite_dataset.csv",index=False)

# Label the feasibility to climbers with differnt climbing routes

In [16]:
from ast import literal_eval

casual_climbers = pd.read_csv("casual_dataset.csv")
skilled_climbers = pd.read_csv("skilled_dataset.csv")
elite_climbers = pd.read_csv("elite_dataset.csv")
routes = pd.read_csv("simulated_routes.csv")
routes["parsed_points"] = routes["points"].apply(literal_eval)

In [17]:
from assign_feasibility import is_route_feasible
from math import dist

casuals = []
skilleds = []
elites = []

group_of_climbers = {
    "casual_climbers": casual_climbers,
    "skilled_climbers": skilled_climbers,
    "elite_climbers": elite_climbers
}

# Label feasibility
for name, category in group_of_climbers.items():
    for i, climber in category.iterrows():
        for j, row in routes.iterrows():
            points = row['parsed_points']
            max_distance = max(dist(points[k], points[k + 1]) for k in range(len(points) - 1))
            sample = {
                "climber_id": f"{name}_{i:03d}",
                "route_id": row["route_id"],
                "feasible": int(is_route_feasible(climber, points)),
                "features": {
                    "height": round(climber["height"], 1),
                    "weight": round(climber["weight"], 1),
                    "ape_index": round(climber["ape_index"], 2),
                    "strength": int(climber["strength"]),
                    "flexibility": int(climber["flexibility"]),
                    "n_points": len(points),
                    "max_hold_distance": round(max_distance, 2)
                }
            }
            if name == "casual_climbers":
                casuals.append(sample)
            elif name == "skilled_climbers":
                skilleds.append(sample)
            else:
                elites.append(sample)

In [21]:
import json

# Output the documents to local data
with open("casuals_feas.json","w") as f:
    json.dump(casuals,f,indent=2)

with open("skilleds_feas.json","w") as f:
    json.dump(skilleds,f,indent=2)

with open("elites_feas.json","w") as f:
    json.dump(elites,f,indent=2)


In [11]:
# from route_parser import pixel_dist

# for j, row in routes.iterrows():
#     points = row['parsed_points']
#     max_distance = max(pixel_dist(points[k], points[k + 1]) for k in range(len(points) - 1))
#     print(max_distance)